_Run the first 2 cells without modifications_

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import get_ipython
from IPython.display import Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
from pathlib import Path
print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))


def add_to_sys_path(this_path, up=False):

    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """

    if up:
        newp = str(Path(this_path).parent)
    else:
        newp = str(Path(this_path))

    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


import numpy as np
import pandas as pd
#pd.set_option("display.max_colwidth", 200)
import matplotlib as mpl
from matplotlib import pyplot as plt
#plt.ion()
plt.style.use('seaborn-v0_8-muted')
from pprint import pprint as pp

def fdir(obj, start_with_str='_', exclude=True):
    """Filtered dir() for method discovery."""
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]


def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    return HTML('<div style="{}">{}</div>'.format(style, title))


def add_div(div_class, div_start, div_text, output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output string, but the cell mode is still in 'code' not 'markdown':
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    The only thing to do is change the cell mode to Markdown.
    If `output_string=False`, the HTML output is displayed in an output cell.
    """

    accepted = ['alert-info', 'alert-warning', 'alert-danger']

    if div_class not in accepted:
        return HTML(f"""<div class="alert"><b>Wrong class:</b> `div_start` is one of {accepted}.
                    </div>""")
    div = f"""<div class="alert {div_class}"><b>{div_start}</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div) #HTML(div)



# autoreload extension
ipython = get_ipython()
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

Python ver: 3.11.5 | packaged by conda-forge | (main, Aug 27 2023, 03:34:09) [GCC 12.3.0]
Python env: mce
Currrent dir: /home/cat/projects/MCCE_Benchmarking/notebooks



In [2]:
add_to_sys_path(Path.cwd(), up=True)

Path added to sys.path: /home/cat/projects/MCCE_Benchmarking


---
---


In [5]:
DATA = Path.cwd().parent.joinpath("data")

PDBS = DATA.joinpath("clean_pdbs")
PDBS
WT = DATA.joinpath("WT_pkas.csv")
WT
Q_BOOK = DATA.joinpath("book.txt")
Q_BOOK
#PROTS = DATA.joinpath("proteins.txt")

PosixPath('/home/cat/projects/MCCE_Benchmarking/data/clean_pdbs')

PosixPath('/home/cat/projects/MCCE_Benchmarking/data/WT_pkas.csv')

PosixPath('/home/cat/projects/MCCE_Benchmarking/data/book.txt')

In [20]:
from benchmark import audit
import subprocess
import getpass
import shutil
import random

In [ ]:
def get_jobs(job_name):
    """
    (base) cat@LABTOP:~/projects$ ps -u
    USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
    cat          9  0.0  0.0   9540  5568 pts/0    Ss   15:20   0:00 -bash
    cat        116  0.0  0.0   9552  5652 pts/1    Ss   15:50   0:01 -bash
    cat        569  0.5  1.3 435456 133116 pts/1   Sl+  17:28   0:09 /home/cat/miniconda3/bin/python3.11 /home/cat/miniconda3/bin/jupyter-lab
    cat        981  0.0  0.0  10460  3252 pts/0    R+   17:57   0:00 ps -u

    (base) cat@LABTOP:~/projects$ pgrep bash
    9
    116
    """
    #pgrep_cmd = f"pgrep {job_name}"
    #poc_id = subprocess.Popen([pgrep_cmd]], stdout=subprocess.PIPE).stdout.readlines()

    lines = subprocess.Popen(["ps", "-u", f"{getpass.getuser()}"], stdout=subprocess.PIPE).stdout.readlines()
    job_uids = [x.decode("ascii").split()[0] for x in lines if x.decode("ascii").find(job_name) > 0]
    job_uids = [x for x in job_uids if x and x != "PID"]

    jobs = []
    for uid in job_uids:
        output = subprocess.Popen(["pwdx", uid], stdout=subprocess.PIPE).stdout.readlines()[0].decode("ascii")
        job = output.split(":")[1].split("/")[-1].strip()
        if job:
            jobs.append(job)
    return jobs

In [35]:
job_name = "bash"
#cmd = f"ps -u {getpass.getuser()}"

cmd = f"pgrep -u {getpass.getuser()} {job_name}"
data = subprocess.check_output(cmd,
                               stderr=subprocess.STDOUT,
                               text=True,
                               shell=True
                              ).splitlines()

print("pgrep data", data)

dirs = []
for uid in data:
    #output = subprocess.Popen(["pwdx", uid], stdout=subprocess.PIPE).stdout.readlines()[0].decode("ascii")
    cmd = f"pwdx {uid}"
    output = subprocess.check_output(cmd,
                                     stderr=subprocess.STDOUT,
                                     text=True,
                                     shell=True
                                     ).strip()
    print(f"pwdx data: {output = }")
    #d = output.split(":")[1].split("/")[-1].strip()
    #if d:
    #    dirs.append(d)


pgrep data ['9', '116']
pwdx data: output = '9: /home/cat/projects/MCCE_Benchmarking/notebooks'
pwdx data: output = '116: /home/cat/projects/MCCE_Benchmarking'


In [19]:
runprm = "run.prm.record"

cmd = f"grep -E 'MONTE_FLIPS|MONTE_RUNS|MONTE_NITER' ./{runprm}| sed -e 's/(//g; s/MONTE_//g; s/)//g'"
data = subprocess.check_output(cmd,
                                stderr=subprocess.STDOUT,
                                text=True,
                                  shell=True
                                  ).splitlines()

FLIPS = int(data[0].split()[0])
RUNS =  int(data[1].split()[0])
NITER = int(data[2].split()[0])
FLIPS, RUNS, NITER

(3, 6, 2000)